# MSc Thesis - Hugo Stam

### Example 02 - Offshore Port System - Barges

<img src="Figures/system_layout_alternative_2B.png" style="width:1000px">

In [1]:
# packages for data handling
import numpy as np
import pandas as pd
import statistics as st

# packages for figures
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
# %matplotlib inline
%matplotlib notebook

import seaborn as sns
sns.set(style="ticks")
# sns.set_style("whitegrid")
# sns.set(style="darkgrid", palette = "pastel")
# sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 1.5})

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# OpenTISim package
from opentisim import container_objects
from opentisim import container_defaults
from opentisim import container_system_offshore_barge

# OpenCLSim package
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# Additional import to save the initialisation of the simulation
import openclsim.savesim as savesim

# Pretty-print a Python object to a stream
from pprint import pprint

# Jupyter Widgets
import ipywidgets as widgets
from IPython.display import HTML

# Scroll to specific cell
from jupyter_helpers.utilities import scroll_to_current_cell

# no warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# iPython Widgests
life_cycle = widgets.IntSlider(value=20, min=10, max=30, step=5, description='Life cycle:')
annual_demand = widgets.IntSlider(value=1000000, min=500000, max=1500000, step=50000, description='Demand:')
offshore_onshore_distance = widgets.IntSlider(value=40, min=20, max=60, step=20, description='Distance:')

container_ship = widgets.Dropdown(options=[('Post Panamax I (6,000 TEU)', 6000),
                                           ('New-Panamax (12,500 TEU)', 12500),
                                           ('ULCS (21,000 TEU)', 21000)], 
                                  value=12500, description='OGV:')

barge = widgets.Dropdown(options=[('Small (200 TEU)', 200), 
                                  ('Medium (250 TEU)', 250),
                                  ('Large (300 TEU)', 300)], 
                         value=250, description='Barge:')

foreshore_slope = widgets.Dropdown(options=[('Gently (3 km/m)', 3.0), 
                                            ('Average (2 km/m)', 2.0),
                                            ('Steep (1 km/m)', 1.0)], 
                                   value=2.0, description='Slope:')

print('\033[1m*** Input Values \033[0m')
display(annual_demand, offshore_onshore_distance, container_ship, barge, 
        foreshore_slope, life_cycle)

*** Input Values 


IntSlider(value=1000000, description='Demand:', max=1500000, min=500000, step=50000)

IntSlider(value=40, description='Distance:', max=60, min=20, step=20)

Dropdown(description='OGV:', index=1, options=(('Post Panamax I (6,000 TEU)', 6000), ('New-Panamax (12,500 TEU…

Dropdown(description='Barge:', index=1, options=(('Small (200 TEU)', 200), ('Medium (250 TEU)', 250), ('Large …

Dropdown(description='Slope:', index=1, options=(('Gently (3 km/m)', 3.0), ('Average (2 km/m)', 2.0), ('Steep …

IntSlider(value=20, description='Life cycle:', max=30, min=10, step=5)

In [3]:
scroll_to_current_cell(preserve=True)

In [4]:
offshore_distance = offshore_onshore_distance.value
demand = annual_demand.value
design_container_ship = container_ship.value
design_barge = barge.value
slope = foreshore_slope.value
lifecycle = life_cycle.value

print('The offshore distance:', offshore_distance, 'km')
print('The annual demand:', demand, 'TEU')
print('The design container ship capacity:', design_container_ship, 'TEU')
print('The design barge capacity:', design_barge, 'TEU')
print('The slope of the foreshore:', slope, 'in km reach per meter water depth')
print('The design life cycle:', lifecycle, 'years')

avg_parcel = design_container_ship / 8

startyear = 2020

The offshore distance: 40 km
The annual demand: 1000000 TEU
The design container ship capacity: 12500 TEU
The design barge capacity: 250 TEU
The slope of the foreshore: 2.0 in km reach per meter water depth
The design life cycle: 20 years


In [5]:
container = demand  # TEU
years = list(range(startyear, startyear+lifecycle+1))

trend_1 = []
trend_2 = []
trend_3 = []

for i in range(lifecycle-4):
    trend_1.append(container +   500_000/15 * i)
    trend_2.append(container + 1_000_000/15 * i)
    trend_3.append(container + 1_500_000/15 * i)

demand_0 = []
demand_1 = []
demand_2 = []
demand_3 = []

for year in years:
    if year < 2025:
        demand_0.append(int(container * 1.00))
        demand_1.append(int(container * 1.00))
        demand_2.append(int(container * 1.00))
        demand_3.append(int(container * 1.00))
        
    elif year < 2030:
        demand_0.append(int(container * 1.00))
        demand_1.append(int(container * 1.17))
        demand_2.append(int(container * 1.33))
        demand_3.append(int(container * 1.50))
    elif year < 2035:
        demand_0.append(int(container * 1.00))
        demand_1.append(int(container * 1.33))
        demand_2.append(int(container * 1.67))
        demand_3.append(int(container * 2.00))
    else:
        demand_0.append(int(container * 1.00))
        demand_1.append(int(container * 1.50))
        demand_2.append(int(container * 2.00))
        demand_3.append(int(container * 2.50))

# # figure 
# plt.figure(figsize=(10,6))

# plt.plot(years[5:lifecycle+1], trend_1, color='tab:green', linestyle='--', zorder=0)
# plt.plot(years[5:lifecycle+1], trend_2, color='tab:blue' , linestyle='--', zorder=0)
# plt.plot(years[5:lifecycle+1], trend_3, color='tab:cyan' , linestyle='--', zorder=0)

# plt.step(years, demand_0, color='tab:olive', linewidth=3, label='base', where='post', zorder=4)
# plt.step(years, demand_1, color='tab:green', linewidth=3, label='scenario 1', where='post', zorder=3)
# plt.step(years, demand_2, color='tab:blue' , linewidth=3, label='scenario 2', where='post', zorder=2)
# plt.step(years, demand_3, color='tab:cyan' , linewidth=3, label='scenario 3', where='post', zorder=1)
    
# plt.xlabel("Date (years)", fontsize='large')

# if lifecycle == 20:
#     plt.xticks([2020, 2025, 2030, 2035, 2040])

# plt.yticks([0, 0.5e6, 1e6, 1.5e6, 2e6, 2.5e6, 3e6],
#            [0, '500,000', '1,000,000', '1,500,000', '2,000,000', '2,500,000', '3,000,000'])
# plt.ylabel('Demand [TEU]', fontsize='large')

# plt.legend(loc='best', title='Growth scenarios', title_fontsize='large', fontsize='large', frameon=False)

# # figure aesthetics
# plt.grid(False, which='major')
# sns.despine(top=True)

# # save fig
# if lifecycle == 20 and demand == 1_000_000:
#     plt.savefig("SaveFigures/growth_scenarios.png")

In [6]:
if lifecycle == 20:
    years = years[0:20]
    demand_0 = demand_0[0:20]
    demand_1 = demand_1[0:20]
    demand_2 = demand_2[0:20]
    demand_3 = demand_3[0:20]
    
if lifecycle == 10:
    years = years[0:10]
    demand_0 = demand_0[0:10]
    demand_1 = demand_1[0:10]
    demand_2 = demand_2[0:10]
    demand_3 = demand_3[0:10]

In [16]:
years = list(range(startyear, startyear + lifecycle))
       
scenario_demand = demand_0
scenario_data={'year': years, 'volume': scenario_demand}
# print(scenario_data)

# instantiate Commodity objects, the inputs for the Commodity class
if design_container_ship == 6000:
    container_defaults.container_data['post_panamax_I_perc'] = 100
    container_defaults.container_data['new_panamax_perc'] = 0
    container_defaults.container_data['ULCS_perc'] = 0

if design_container_ship == 12500:
    container_defaults.container_data['post_panamax_I_perc'] = 0
    container_defaults.container_data['new_panamax_perc'] = 100
    container_defaults.container_data['ULCS_perc'] = 0
    
if design_container_ship == 21000:
    container_defaults.container_data['post_panamax_I_perc'] = 0
    container_defaults.container_data['new_panamax_perc'] = 0
    container_defaults.container_data['ULCS_perc'] = 100
    
# print(container_defaults.container_data)

# ladens
container = container_objects.Commodity(**container_defaults.container_data)
container.scenario_data = pd.DataFrame(data=scenario_data)
# print(container)
# print(container.scenario_data)

# combine
demand = [container]

In [8]:
# instantiate vessels
fully_cellular = container_objects.Vessel(**container_defaults.fully_cellular_data)
panamax = container_objects.Vessel(**container_defaults.panamax_data)
panamax_max = container_objects.Vessel(**container_defaults.panamax_max_data)
post_panamax_I = container_objects.Vessel(**container_defaults.post_panamax_I_data)
post_panamax_II = container_objects.Vessel(**container_defaults.post_panamax_II_data)
new_panamax = container_objects.Vessel(**container_defaults.new_panamax_data)
VLCS = container_objects.Vessel(**container_defaults.VLCS_data)
ULCS = container_objects.Vessel(**container_defaults.ULCS_data)

vessels = [fully_cellular, panamax, panamax_max, post_panamax_I, post_panamax_II, new_panamax, VLCS, ULCS] 

In [17]:
OffshoreBarge = container_system_offshore_barge.System(
                startyear = startyear,
                lifecycle = lifecycle,
                stack_equipment = 'rmg',
                laden_stack = 'rmg',
                elements = demand + vessels,
                crane_type_defaults = container_defaults.sts_crane_data,
                offshore_distance = offshore_distance,
                barge_type = 'medium',
                foreshore_slope = slope,
                bathymetry_factor = 0.80,
                allowable_berth_occupancy = 0.70,
                laden_perc=0.80, 
                reefer_perc=0.10,
                empty_perc=0.075, 
                oog_perc=0.025,
                transhipment_ratio = 0.0,
                debug = True)

# Start simulation

In [18]:
# scroll_to_current_cell(preserve=True)
OffshoreBarge.simulate()

*** nr of OGV berths: 1
     >> max vessel length: 366
     >> quay length 396 m
     >> quay depth 19 m


,Year,Offshore Capex,Offshore Insurance
0,2020,27760383.6,0.00
1,2021,18506922.4,0.00
2,2022,0.0,405250.56
3,2023,0.0,405250.56
4,2024,0.0,405250.56


,Year,Offshore Capex,Offshore Insurance,Offshore Labour
0,2020,0.0,0.0,0.0
1,2021,11500000.0,0.0,0.0
2,2022,0.0,100000.0,687500.0
3,2023,0.0,100000.0,687500.0
4,2024,0.0,100000.0,687500.0


*** nr of OGV berths: 1


,Year,Offshore Capex,Offshore Insurance,Offshore Labour
0,2020,0.0,0.0,0.0
1,2021,11500000.0,0.0,0.0
2,2022,0.0,100000.0,687500.0
3,2023,0.0,100000.0,687500.0
4,2024,0.0,100000.0,687500.0


...

,Year,Ocean Transport
0,2020,0.0
1,2021,0.0
2,2022,0.0
3,2023,0.0
4,2024,0.0


,Year,Ocean Transport
0,2020,0.0
1,2021,0.0
2,2022,0.0
3,2023,0.0
4,2024,0.0


,Year,Ocean Transport
0,2020,0.0
1,2021,0.0
2,2022,0.0
3,2023,0.0
4,2024,0.0


check offshore_land_use 37.4134906 ha
check onshore_land_use 19.914040600000003 ha


,Years,Capex,Opex,PV,cum-PV
0,2020,7.675239e+08,0.000000e+00,7.675239e+08,7.675239e+08
1,2021,5.608752e+08,3.016259e+07,5.910378e+08,1.358562e+09
2,2022,0.000000e+00,1.406622e+08,1.406622e+08,1.499224e+09
3,2023,0.000000e+00,1.296121e+08,1.296121e+08,1.628836e+09
4,2024,0.000000e+00,1.193644e+08,1.193644e+08,1.748200e+09
5,2025,0.000000e+00,1.098636e+08,1.098636e+08,1.858064e+09
6,2026,0.000000e+00,1.010578e+08,1.010578e+08,1.959122e+09
7,2027,0.000000e+00,9.289863e+07,9.289863e+07,2.052020e+09
8,2028,0.000000e+00,8.534108e+07,8.534108e+07,2.137361e+09
9,2029,0.000000e+00,7.834304e+07,7.834304e+07,2.215704e+09


In [19]:
demand = annual_demand.value

cash_flows_df, cash_flows_WACC_real_df = OffshoreBarge.add_cashflow_elements()
PV, PV_df = OffshoreBarge.net_present_value(False)

if lifecycle == 20 and scenario_demand == demand_0:
    if demand == 1000000:
        if offshore_distance == 20:  
            if design_container_ship == 6000:
                PV_alt2_20_6000_1000000 = PV
                %store PV_alt2_20_6000_1000000

            if design_container_ship == 12500:
                PV_alt2_20_12500_1000000 = PV
                %store PV_alt2_20_12500_1000000

            if design_container_ship == 21000:
                PV_alt2_20_21000_1000000 = PV
                %store PV_alt2_20_21000_1000000

        if offshore_distance == 40:  
            if design_container_ship == 6000:
                PV_alt2_40_6000_1000000 = PV
                %store PV_alt2_40_6000_1000000

            if design_container_ship == 12500:
                PV_alt2_40_12500_1000000 = PV
                %store PV_alt2_40_12500_1000000

            if design_container_ship == 21000:
                PV_alt2_40_21000_1000000 = PV
                %store PV_alt2_40_21000_1000000

        if offshore_distance == 60:  
            if design_container_ship == 6000:
                PV_alt2_60_6000_1000000 = PV
                %store PV_alt2_60_6000_1000000

            if design_container_ship == 12500:
                PV_alt2_60_12500_1000000 = PV
                %store PV_alt2_60_12500_1000000

            if design_container_ship == 21000:
                PV_alt2_60_21000_1000000 = PV
                %store PV_alt2_60_21000_1000000

Stored 'PV_alt2_40_12500_1000000' (float64)


In [20]:
# demand
if lifecycle == 20 and scenario_demand == demand_0 and offshore_distance == 40 and design_container_ship == 12500:
    if demand == 500_000:
        PV_alt2_40_12500_500000 = PV
        %store PV_alt2_40_12500_500000
    if demand == 1_000_000:
        PV_alt2_40_12500_1000000 = PV
        %store PV_alt2_40_12500_1000000
    if demand == 1_500_000:
        PV_alt2_40_12500_1500000 = PV
        %store PV_alt2_40_12500_1500000

Stored 'PV_alt2_40_12500_1000000' (float64)


In [21]:
# growing demand
if lifecycle == 20 and offshore_distance == 40 and design_container_ship == 12500 and slope == 2.0:
    if scenario_demand == demand_0:
        PV_alt2_40_12500_1000000_growth0 = PV
        %store PV_alt2_40_12500_1000000_growth0
    if scenario_demand == demand_1:
        PV_alt2_40_12500_1000000_growth1 = PV
        %store PV_alt2_40_12500_1000000_growth1
    if scenario_demand == demand_2:
        PV_alt2_40_12500_1000000_growth2 = PV
        %store PV_alt2_40_12500_1000000_growth2
    if scenario_demand == demand_3:
        PV_alt2_40_12500_1000000_growth3 = PV
        %store PV_alt2_40_12500_1000000_growth3

Stored 'PV_alt2_40_12500_1000000_growth0' (float64)


In [22]:
# slope in km reach per meter water depth
if lifecycle == 20 and offshore_distance == 40 and design_container_ship == 12500 and scenario_demand == demand_0:
    if slope == 3.0:
        PV_alt2_40_12500_1000000_slope1 = PV
        %store PV_alt2_40_12500_1000000_slope1
    if slope == 2.0:
        PV_alt2_40_12500_1000000_slope2 = PV
        %store PV_alt2_40_12500_1000000_slope2
    if slope == 1.0:
        PV_alt2_40_12500_1000000_slope3 = PV
        %store PV_alt2_40_12500_1000000_slope3

Stored 'PV_alt2_40_12500_1000000_slope2' (float64)
